<a href="https://colab.research.google.com/github/MdShahadatHossainbd/ML-Project/blob/main/1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import all the necessary packages

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
from matplotlib import style
style.use('ggplot')

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#train = pd.read_csv('/content/gdrive/MyDrive/archive/drugsComTrain_raw.csv') #train data
#test = pd.read_csv('/content/gdrive/MyDrive/archive/drugsComTest_raw.csv') #test data

In [4]:
def load_dataset(file_path, num_samples):
    df = pd.read_csv(file_path, usecols=[3, 4], nrows=num_samples)
    df.columns = ['review', 'rating']

    text = df['review'].tolist()
    text = [str(t).encode('ascii', 'replace') for t in text]
    text = np.array(text, dtype=object)[:]
    
    labels = df['rating'].tolist()
    labels = [1 if i>=7 else 0 if i>=5 else -1 for i in labels]
    labels = np.array(pd.get_dummies(labels), dtype=int)[:] 

    return labels, text

In [5]:
tmp_labels, tmp_text = load_dataset('/content/gdrive/MyDrive/archive/drugsComTrain_raw.csv', 568454)
tmp_text.shape

(161297,)

In [6]:
test_labels, test_text = load_dataset('/content/gdrive/MyDrive/archive/drugsComTest_raw.csv',60000)
test_text.shape

(53766,)

In [9]:
#import the libraries for pre-processing
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stops = set(stopwords.words('english')) #english stopwords

stemmer = SnowballStemmer('english') #SnowballStemmer

def review_to_words(raw_review):
    # 1. Delete HTML 
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 2. Make a space
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 3. lower letters
    words = letters_only.lower().split()
    # 5. Stopwords 
    meaningful_words = [w for w in words if not w in stops]
    # 6. Stemming
    stemming_words = [stemmer.stem(w) for w in meaningful_words]
    # 7. space join words
    #return( ' '.join(stemming_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
import tensorflow as tf
import tensorflow_hub as hub

In [12]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1", output_shape=[50], 
                           input_shape=[], dtype=tf.string, name='input', trainable=False)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(1024, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(3, activation='softmax', name='output'))
model.compile(loss='categorical_crossentropy',
                  optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (KerasLayer)           (None, 128)               124642688 
_________________________________________________________________
dense (Dense)                (None, 1024)              132096    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
output (Dense)               (None, 3)                 3075      
Total params: 124,777,859
Trainable params: 135,171
Non-trainable params: 124,642,688
_________________________________________________________________


In [13]:
print("Training the model ...")
history = model.fit(test_text,test_labels, batch_size = 128, epochs=50, verbose=1,
              validation_data=(test_text,test_labels),)

Training the model ...
Epoch 1/50
421/421 [==============================] - 9s 21ms/step - loss: 0.7994 - accuracy: 0.6710 - val_loss: 0.7369 - val_accuracy: 0.6965
Epoch 2/50
421/421 [==============================] - 11s 27ms/step - loss: 0.7348 - accuracy: 0.6992 - val_loss: 0.7160 - val_accuracy: 0.7082
Epoch 3/50
421/421 [==============================] - 12s 30ms/step - loss: 0.7228 - accuracy: 0.7066 - val_loss: 0.7006 - val_accuracy: 0.7166
Epoch 4/50
421/421 [==============================] - 11s 26ms/step - loss: 0.7031 - accuracy: 0.7149 - val_loss: 0.6888 - val_accuracy: 0.7247
Epoch 5/50
421/421 [==============================] - 12s 28ms/step - loss: 0.6881 - accuracy: 0.7229 - val_loss: 0.6722 - val_accuracy: 0.7321
Epoch 6/50
421/421 [==============================] - 12s 29ms/step - loss: 0.6830 - accuracy: 0.7262 - val_loss: 0.6712 - val_accuracy: 0.7245
Epoch 7/50
421/421 [==============================] - 11s 25ms/step - loss: 0.6617 - accuracy: 0.7330 - val_loss: 

In [14]:
model.predict(['im feeling bad'])

array([[0.3478613 , 0.13545886, 0.5166799 ]], dtype=float32)